# Quantum Transport Simulations with Kwant

This notebook demonstrates quantum transport calculations in mesoscopic systems and materials using Kwant.

In [ ]:
import kwant
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import pyplot
import scipy.sparse.linalg as sla
from typing import Tuple, Optional, Callable

## 1. Basic Quantum Wire

In [ ]:
def make_wire(W: int = 10, L: int = 30) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1, norbs=1)
    
    syst = kwant.Builder()
    
    syst[(lat(x, y) for x in range(L) for y in range(W))] = 4
    syst[lat.neighbors()] = -1
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead)
    
    lead[(lat(0, y) for y in range(W))] = 4
    lead[lat.neighbors()] = -1
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

wire = make_wire()
kwant.plot(wire)
plt.title('Quantum Wire System')
plt.show()

energies = np.linspace(0, 4, 100)
conductances = []

for energy in energies:
    smatrix = kwant.smatrix(wire, energy)
    conductances.append(smatrix.transmission(1, 0))

plt.figure(figsize=(8, 4))
plt.plot(energies, conductances)
plt.xlabel('Energy')
plt.ylabel('Conductance (e²/h)')
plt.title('Conductance Quantization in Quantum Wire')
plt.grid(True)
plt.show()

## 2. Graphene Nanoribbon

In [ ]:
def make_graphene_ribbon(W: int = 5, L: int = 10) -> kwant.system.FiniteSystem:
    graphene = kwant.lattice.honeycomb(a=1, norbs=1)
    a, b = graphene.sublattices
    
    def ribbon_shape(pos):
        x, y = pos
        return 0 <= x < L and 0 <= y < W
    
    syst = kwant.Builder()
    syst[graphene.shape(ribbon_shape, (0, 0))] = 0
    syst[graphene.neighbors()] = -1
    
    def lead_shape(pos):
        x, y = pos
        return 0 <= y < W
    
    sym_lead = kwant.TranslationalSymmetry(graphene.vec((-1, 0)))
    lead = kwant.Builder(sym_lead)
    
    lead[graphene.shape(lead_shape, (0, 0))] = 0
    lead[graphene.neighbors()] = -1
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

def plot_bandstructure(lead, momenta):
    bands = kwant.physics.Bands(lead, momenta=momenta)
    energies = [bands(k) for k in momenta]
    
    plt.figure(figsize=(8, 5))
    plt.plot(momenta, energies)
    plt.xlabel('k')
    plt.ylabel('Energy')
    plt.title('Band Structure of Graphene Nanoribbon')
    plt.grid(True)
    plt.show()

graphene_system = make_graphene_ribbon(W=10, L=20)
kwant.plot(graphene_system)
plt.title('Graphene Nanoribbon')
plt.show()

momenta = np.linspace(-np.pi, np.pi, 200)
plot_bandstructure(graphene_system.leads[0], momenta)

## 3. Quantum Hall Effect

In [ ]:
def make_hall_bar(W: int = 20, L: int = 40, B: float = 0.1) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1)
    
    def hopping(site1, site2, B):
        x1, y1 = site1.pos
        x2, y2 = site2.pos
        phase = B * (x1 - x2) * (y1 + y2) / 2
        return -np.exp(1j * phase)
    
    syst = kwant.Builder()
    
    syst[(lat(x, y) for x in range(L) for y in range(W))] = 4
    syst[lat.neighbors()] = hopping
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead)
    
    def lead_hopping(site1, site2, B):
        return -1
    
    lead[(lat(0, y) for y in range(W))] = 4
    lead[lat.neighbors()] = lead_hopping
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    sym_lead_top = kwant.TranslationalSymmetry((0, 1))
    lead_top = kwant.Builder(sym_lead_top)
    lead_top[(lat(x, 0) for x in range(L//3, 2*L//3))] = 4
    lead_top[lat.neighbors()] = lead_hopping
    syst.attach_lead(lead_top)
    
    sym_lead_bottom = kwant.TranslationalSymmetry((0, -1))
    lead_bottom = kwant.Builder(sym_lead_bottom)
    lead_bottom[(lat(x, 0) for x in range(L//3, 2*L//3))] = 4
    lead_bottom[lat.neighbors()] = lead_hopping
    syst.attach_lead(lead_bottom.reversed())
    
    return syst.finalized()

B_values = np.linspace(0, 0.4, 50)
hall_conductances = []

for B in B_values:
    hall_system = make_hall_bar(B=B)
    smatrix = kwant.smatrix(hall_system, 2.0, params=dict(B=B))
    hall_conductances.append(smatrix.transmission(2, 0))

plt.figure(figsize=(10, 5))
plt.plot(B_values, hall_conductances)
plt.xlabel('Magnetic Field B')
plt.ylabel('Hall Conductance (e²/h)')
plt.title('Quantum Hall Effect')
plt.grid(True)
plt.show()

## 4. Topological Insulator Edge States

In [ ]:
def make_bhz_system(W: int = 20, L: int = 30, M: float = 1.0) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1, norbs=4)
    
    def onsite(site, M):
        return (M - 4) * np.kron(np.eye(2), np.array([[1, 0], [0, -1]]))
    
    def hopping_x(site1, site2):
        return np.kron(np.eye(2), np.array([[1, 0], [0, -1]])) + \
               1j * np.kron(np.array([[0, 1], [1, 0]]), np.array([[0, -1j], [1j, 0]]))
    
    def hopping_y(site1, site2):
        return np.kron(np.eye(2), np.array([[1, 0], [0, -1]])) + \
               1j * np.kron(np.array([[0, -1j], [1j, 0]]), np.array([[0, -1j], [1j, 0]]))
    
    syst = kwant.Builder()
    
    syst[(lat(x, y) for x in range(L) for y in range(W))] = onsite
    
    syst[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    syst[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead)
    
    lead[(lat(0, y) for y in range(W))] = onsite
    lead[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    lead[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

M_values = np.linspace(-2, 2, 100)
edge_conductances = []

for M in M_values:
    ti_system = make_bhz_system(M=M)
    smatrix = kwant.smatrix(ti_system, 0.0, params=dict(M=M))
    edge_conductances.append(smatrix.transmission(1, 0))

plt.figure(figsize=(10, 5))
plt.plot(M_values, edge_conductances)
plt.xlabel('Mass Parameter M')
plt.ylabel('Edge State Conductance (e²/h)')
plt.title('Topological Phase Transition')
plt.axvline(x=0, color='r', linestyle='--', label='Phase Transition')
plt.legend()
plt.grid(True)
plt.show()

## 5. Anderson Localization

In [ ]:
def make_disordered_wire(W: int = 10, L: int = 100, disorder: float = 0.5) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1)
    
    def onsite(site, disorder):
        return 4 + disorder * (np.random.random() - 0.5)
    
    syst = kwant.Builder()
    
    syst[(lat(x, y) for x in range(L) for y in range(W))] = onsite
    syst[lat.neighbors()] = -1
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead)
    
    lead[(lat(0, y) for y in range(W))] = 4
    lead[lat.neighbors()] = -1
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

def calculate_localization_length(disorder: float, n_samples: int = 20) -> float:
    conductances = []
    
    for _ in range(n_samples):
        wire = make_disordered_wire(disorder=disorder)
        smatrix = kwant.smatrix(wire, 2.0, params=dict(disorder=disorder))
        conductances.append(smatrix.transmission(1, 0))
    
    return np.mean(conductances), np.std(conductances)

disorder_values = np.linspace(0, 5, 20)
mean_conductances = []
std_conductances = []

for disorder in disorder_values:
    mean_g, std_g = calculate_localization_length(disorder, n_samples=10)
    mean_conductances.append(mean_g)
    std_conductances.append(std_g)

plt.figure(figsize=(10, 5))
plt.errorbar(disorder_values, mean_conductances, yerr=std_conductances, fmt='o-')
plt.xlabel('Disorder Strength')
plt.ylabel('Average Conductance (e²/h)')
plt.title('Anderson Localization')
plt.grid(True)
plt.show()

## 6. Superconducting Junction

In [ ]:
def make_sns_junction(W: int = 10, L_n: int = 20, Delta: float = 0.1) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1, norbs=2)
    
    tau_x = np.array([[0, 1], [1, 0]])
    tau_y = np.array([[0, -1j], [1j, 0]])
    tau_z = np.array([[1, 0], [0, -1]])
    
    def onsite_normal(site):
        return 4 * tau_z
    
    def onsite_sc(site, Delta):
        return 4 * tau_z + Delta * tau_x
    
    def hopping(site1, site2):
        return -tau_z
    
    syst = kwant.Builder()
    
    syst[(lat(x, y) for x in range(L_n) for y in range(W))] = onsite_normal
    syst[lat.neighbors()] = hopping
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead_sc = kwant.Builder(sym_lead, conservation_law=-tau_z)
    
    lead_sc[(lat(0, y) for y in range(W))] = onsite_sc
    lead_sc[lat.neighbors()] = hopping
    
    syst.attach_lead(lead_sc)
    syst.attach_lead(lead_sc.reversed())
    
    return syst.finalized()

def andreev_reflection(energy: float, Delta: float = 0.1) -> float:
    sns = make_sns_junction(Delta=Delta)
    smatrix = kwant.smatrix(sns, energy, params=dict(Delta=Delta))
    
    r_ee = smatrix.submatrix((0, 0), (0, 0))
    r_he = smatrix.submatrix((0, 1), (0, 0))
    
    normal_reflection = np.sum(np.abs(r_ee)**2)
    andreev_reflection = np.sum(np.abs(r_he)**2)
    
    return andreev_reflection / (normal_reflection + andreev_reflection + 1e-10)

energies = np.linspace(0, 0.2, 100)
andreev_probs = [andreev_reflection(E, Delta=0.1) for E in energies]

plt.figure(figsize=(10, 5))
plt.plot(energies, andreev_probs)
plt.xlabel('Energy')
plt.ylabel('Andreev Reflection Probability')
plt.title('Andreev Reflection at Superconductor-Normal Interface')
plt.axvline(x=0.1, color='r', linestyle='--', label='Δ = 0.1')
plt.legend()
plt.grid(True)
plt.show()

## 7. Aharonov-Bohm Ring

In [ ]:
def make_ab_ring(r_inner: int = 10, r_outer: int = 15, flux: float = 0) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1)
    
    def ring_shape(pos):
        x, y = pos
        r = np.sqrt(x**2 + y**2)
        return r_inner <= r <= r_outer
    
    def hopping_phase(site1, site2, flux):
        x1, y1 = site1.pos
        x2, y2 = site2.pos
        
        phase = flux * (x1 * y2 - x2 * y1) / (2 * np.pi * r_inner**2)
        return -np.exp(1j * phase)
    
    syst = kwant.Builder()
    
    syst[lat.shape(ring_shape, (0, 0))] = 4
    syst[lat.neighbors()] = hopping_phase
    
    def lead_shape(pos):
        x, y = pos
        return abs(y) <= 2
    
    sym_lead_left = kwant.TranslationalSymmetry((-1, 0))
    lead_left = kwant.Builder(sym_lead_left)
    lead_left[lat.shape(lead_shape, (-r_outer, 0))] = 4
    lead_left[lat.neighbors()] = -1
    
    sym_lead_right = kwant.TranslationalSymmetry((1, 0))
    lead_right = kwant.Builder(sym_lead_right)
    lead_right[lat.shape(lead_shape, (r_outer, 0))] = 4
    lead_right[lat.neighbors()] = -1
    
    syst.attach_lead(lead_left)
    syst.attach_lead(lead_right)
    
    return syst.finalized()

flux_values = np.linspace(0, 4 * np.pi, 200)
ab_conductances = []

for flux in flux_values:
    ab_ring = make_ab_ring(flux=flux)
    smatrix = kwant.smatrix(ab_ring, 2.0, params=dict(flux=flux))
    ab_conductances.append(smatrix.transmission(1, 0))

plt.figure(figsize=(12, 5))
plt.plot(flux_values / np.pi, ab_conductances)
plt.xlabel('Flux (π)')
plt.ylabel('Conductance (e²/h)')
plt.title('Aharonov-Bohm Oscillations')
plt.grid(True)
plt.show()

## 8. Spin-Orbit Coupling Effects

In [ ]:
def make_rashba_wire(W: int = 10, L: int = 30, alpha: float = 0.5) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1, norbs=2)
    
    sigma_0 = np.eye(2)
    sigma_x = np.array([[0, 1], [1, 0]])
    sigma_y = np.array([[0, -1j], [1j, 0]])
    sigma_z = np.array([[1, 0], [0, -1]])
    
    def onsite(site, B_z=0):
        return 4 * sigma_0 + B_z * sigma_z
    
    def hopping_x(site1, site2, alpha):
        return -sigma_0 + 1j * alpha * sigma_y
    
    def hopping_y(site1, site2, alpha):
        return -sigma_0 - 1j * alpha * sigma_x
    
    syst = kwant.Builder()
    
    syst[(lat(x, y) for x in range(L) for y in range(W))] = onsite
    
    syst[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    syst[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead, conservation_law=sigma_z)
    
    lead[(lat(0, y) for y in range(W))] = onsite
    lead[kwant.builder.HoppingKind((1, 0), lat, lat)] = hopping_x
    lead[kwant.builder.HoppingKind((0, 1), lat, lat)] = hopping_y
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

def spin_resolved_conductance(alpha: float, B_z: float = 0) -> dict:
    rashba_system = make_rashba_wire(alpha=alpha)
    smatrix = kwant.smatrix(rashba_system, 2.0, params=dict(alpha=alpha, B_z=B_z))
    
    G_uu = smatrix.transmission((1, 0), (0, 0))
    G_ud = smatrix.transmission((1, 0), (0, 1))
    G_du = smatrix.transmission((1, 1), (0, 0))
    G_dd = smatrix.transmission((1, 1), (0, 1))
    
    return {'up-up': G_uu, 'up-down': G_ud, 'down-up': G_du, 'down-down': G_dd}

alpha_values = np.linspace(0, 1, 50)
spin_conductances = {key: [] for key in ['up-up', 'up-down', 'down-up', 'down-down']}

for alpha in alpha_values:
    G_spin = spin_resolved_conductance(alpha)
    for key in spin_conductances:
        spin_conductances[key].append(G_spin[key])

plt.figure(figsize=(12, 8))
for i, (key, values) in enumerate(spin_conductances.items()):
    plt.subplot(2, 2, i+1)
    plt.plot(alpha_values, values)
    plt.xlabel('Rashba Coupling α')
    plt.ylabel('Conductance (e²/h)')
    plt.title(f'Spin {key} Conductance')
    plt.grid(True)

plt.tight_layout()
plt.show()

## 9. Carbon Nanotube

In [ ]:
def make_cnt(n: int = 5, m: int = 5, length: int = 20) -> kwant.system.FiniteSystem:
    def nanotube_shape(pos):
        x, y, z = pos
        return 0 <= z < length
    
    cnt = kwant.lattice.general([(1, 0, 0), 
                                 (0.5, np.sqrt(3)/2, 0),
                                 (0, 0, 1)], 
                                [(0, 0, 0), (0.5, 1/(2*np.sqrt(3)), 0)],
                                norbs=1)
    
    syst = kwant.Builder(kwant.TranslationalSymmetry((0, 0, 1)))
    
    syst[cnt.shape(nanotube_shape, (0, 0, 0))] = 0
    syst[cnt.neighbors()] = -1
    
    def lead_shape(pos):
        x, y, z = pos
        return True
    
    lead = kwant.Builder(kwant.TranslationalSymmetry((0, 0, -1)))
    lead[cnt.shape(lead_shape, (0, 0, 0))] = 0
    lead[cnt.neighbors()] = -1
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

cnt_system = make_cnt()

energies = np.linspace(-3, 3, 200)
dos = []

for energy in energies:
    try:
        smatrix = kwant.smatrix(cnt_system, energy)
        dos.append(smatrix.transmission(0, 0))
    except:
        dos.append(0)

plt.figure(figsize=(10, 5))
plt.plot(energies, dos)
plt.xlabel('Energy')
plt.ylabel('Density of States')
plt.title('Carbon Nanotube Electronic Structure')
plt.grid(True)
plt.show()

## 10. Quantum Point Contact

In [ ]:
def make_qpc(W0: int = 30, W1: int = 5, L: int = 20) -> kwant.system.FiniteSystem:
    lat = kwant.lattice.square(a=1)
    
    def qpc_shape(pos):
        x, y = pos
        if abs(x) > L:
            return abs(y) < W0 / 2
        else:
            w = W0 / 2 - (W0 - W1) / 2 * (1 - np.cos(np.pi * x / L))
            return abs(y) < w
    
    def potential(site, V0):
        x, y = site.pos
        if abs(x) < L:
            return 4 + V0 * np.exp(-(x**2 + y**2) / (L/3)**2)
        return 4
    
    syst = kwant.Builder()
    
    syst[lat.shape(qpc_shape, (0, 0))] = potential
    syst[lat.neighbors()] = -1
    
    sym_lead = kwant.TranslationalSymmetry((-1, 0))
    lead = kwant.Builder(sym_lead)
    
    lead[(lat(0, y) for y in range(-W0//2, W0//2))] = 4
    lead[lat.neighbors()] = -1
    
    syst.attach_lead(lead)
    syst.attach_lead(lead.reversed())
    
    return syst.finalized()

qpc = make_qpc()
kwant.plot(qpc)
plt.title('Quantum Point Contact Geometry')
plt.show()

V0_values = np.linspace(-2, 2, 100)
qpc_conductances = []

for V0 in V0_values:
    smatrix = kwant.smatrix(qpc, 2.0, params=dict(V0=V0))
    qpc_conductances.append(smatrix.transmission(1, 0))

plt.figure(figsize=(10, 5))
plt.plot(V0_values, qpc_conductances)
plt.xlabel('Gate Voltage V0')
plt.ylabel('Conductance (e²/h)')
plt.title('Quantum Point Contact Conductance')
plt.grid(True)
plt.show()